# Задача на 6-ом шаге урока

**⚖️ Взвешенный блендинг**

Задание: У тебя есть информация о точности модели на лидерборде (accuracy_score). Рассчитай веса, с которыми нужно будет взвесить предсказания моделей, чтобы они были пропорциональны accuracy_score, а их сумма равнялась единице.

In [7]:
def input():
    return """3 
0.3 0.5 0.4"""

Примечание: Ввод и вывод уже написан за вас.

In [8]:
_, *accuracies = map(float, input().split())
weights = [round(acc / sum(accuracies), 7) for acc in accuracies]

print(*weights)

0.25 0.4166667 0.3333333


Sample Input:

3

0.3 0.5 0.4

Sample Output:

0.25 0.4166667 0.3333333

# Задача на 7-ом шаге урока

**🏄‍♂️ Hard voting**

Датасет с прогнозами моделей тут:

In [9]:
path = 'https://stepik.org/media/attachments/lesson/825510/make_soft_blend.csv'

In [10]:
import pandas as pd

In [11]:
pd.read_csv(path).head()

,car_id,target_class,model_1,model_2,model_3,model_4,model_5
0,car_0,NaN,electro_bug,gear_stick,electro_bug,electro_bug,engine_ignition
1,car_1,NaN,engine_check,engine_overheat,electro_bug,engine_overheat,engine_overheat
2,car_2,NaN,another_bug,electro_bug,engine_overheat,engine_overheat,gear_stick
3,car_3,NaN,electro_bug,electro_bug,electro_bug,wheel_shake,electro_bug
4,car_4,NaN,engine_ignition,another_bug,engine_check,engine_check,engine_check


In [ ]:
import pandas as pd
from collections import Counter
from itertools import product


def func(values, sort_values=False, num_pos=0):
    counter = Counter(values)
    most_common = counter.most_common()[0][1]
    keys = [key for key, val in counter.items() if val == most_common]
    if sort_values:
        keys = sorted(keys)
    return keys[min(num_pos, len(keys) - 1)]


# Нахождение моды по строкам
df['target_class'] = df.filter(like='model_', axis=1).mode(axis=1)[0]
df[['car_id', 'target_class']].to_csv('submit_6_1_7_V1.csv', index=False)

df['target_class'] = df.filter(like='model_', axis=1).apply(
    lambda row: row.value_counts().index[0], axis=1)
df[['car_id', 'target_class']].to_csv('submit_6_1_7_V2.csv', index=False)

for sort_, n_pos in product((0, 1), (range(5))):
    df['target_class'] = df.filter(like='model_', axis=1).apply(
        lambda row: func(row.values, sort_values=False, num_pos=n_pos), axis=1)
    df[['car_id', 'target_class']].to_csv(f'submit_6_1_7_V3_{sort_}_{n_pos}.csv', index=False)

Файл с решением должен выглядет так с заполненными значениями target_class

In [12]:
path = 'https://stepik.org/media/attachments/lesson/825510/sb_sample_submission.csv'

In [13]:
pd.read_csv(path).head()

,car_id,target_class
0,car_0,NaN
1,car_1,NaN
2,car_2,NaN
3,car_3,NaN
4,car_4,NaN


# Задача на 8-ом шаге урока

**🤝Soft Voting**

Датасета с предсказаниями поломок машин от разных моделей:

In [ ]:
'https://stepik.org/media/attachments/lesson/825510/catboost_preds.csv'

In [ ]:
'https://stepik.org/media/attachments/lesson/825510/lgbm_preds.csv'

In [ ]:
'https://stepik.org/media/attachments/lesson/825510/xgb_preds.csv'

Ваша задача  — сблендить предсказания этих трех моделей при помощи методики мягкого голосования (soft voting). В качестве ответа отправьте файл, в котором для каждой строчки будет предсказанный ответ, записанный в колонку soft_preds. Названия ответа можно брать из названия соответствующей колонки, обрезая _probа. Порядок строк в высылаемом решении должен быть таким же, как и в файлах с предсказаниями моделей.



In [ ]:
import pandas as pd

summits = ('https://stepik.org/media/attachments/lesson/825510/catboost_preds.csv',
           'https://stepik.org/media/attachments/lesson/825510/lgbm_preds.csv',
           'https://stepik.org/media/attachments/lesson/825510/xgb_preds.csv',
           )
dfs = [pd.read_csv(submit) for submit in summits]
result_data = {col: sum(df[col] for df in dfs if col in df.columns) for col in dfs[0].columns}
df = pd.DataFrame(result_data)
df.columns = [col.replace('_proba', '') for col in df.columns]
df['soft_preds'] = df.idxmax(axis=1)
df[['soft_preds']].to_csv('soft_preds.csv')

# Задача на 10-ом шаге урока

**Напишите свой Rank Averaging**

Реализуйте процедуру нормализации вероятностей, как описывалось в предыдущем шаге (https://stepik.org/lesson/825510/step/9?unit=829017). Для создания тестового датасета можно воспользоваться кодом:

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(42)
proba = np.random.random(100)
df = pd.DataFrame({'id': np.arange(len(proba)), 'proba': proba})
df

,id,proba
0,0,0.374540
1,1,0.950714
2,2,0.731994
3,3,0.598658
4,4,0.156019
...,...,...
95,95,0.493796
96,96,0.522733
97,97,0.427541
98,98,0.025419


Вам нужно посчитать отнормированные предсказания по инструкции из предыдущего степа. Финальный ответ запишите в столбец normalized_proba

In [16]:
import numpy as np
import pandas as pd

df = pd.read_csv('https://stepik.org/media/attachments/lesson/825510/make_hard_blend.csv')
df['proba'] = df['model_1']

# V1
df['normalized_proba'] = df['proba'].rank() / df['proba'].nunique()

# V2
df['normalized_proba'] = df['proba'].rank(pct=True)

# Задача на 11-ом шаге урока

**Rank Averaging для 3-х моделей**

В этот раз вам даны вероятности сразу для трех моделей -  proba1, proba2, proba3. Для создания тестового датасета можно использовать код:




In [ ]:
import numpy as np
import pandas as pd
np.random.seed(42)
df = pd.DataFrame({'id': np.arange(100),
                   'proba1': np.random.random(100),
                   'proba2': np.random.random(100),
                   'proba3': np.random.random(100)})

In [ ]:
df

,id,proba1,proba2,proba3
0,0,0.374540,0.031429,0.642032
1,1,0.950714,0.636410,0.084140
2,2,0.731994,0.314356,0.161629
3,3,0.598658,0.508571,0.898554
4,4,0.156019,0.907566,0.606429
...,...,...,...,...
95,95,0.493796,0.349210,0.522243
96,96,0.522733,0.725956,0.769994
97,97,0.427541,0.897110,0.215821
98,98,0.025419,0.887086,0.622890


Посчитайте для каждой модели ее ранг (как в прошлом степе) и потом запишите среднее значение этих трех рангов в колонку mean_rank.

In [18]:
import numpy as np
import pandas as pd

df = pd.read_csv('https://stepik.org/media/attachments/lesson/825510/make_hard_blend.csv')
df.columns = [col.replace('model_', 'proba') for col in df.columns]

# V1
df['mean_rank'] = df.filter(like='proba', axis=1).rank(pct=True).mean(axis=1)